# `seqp`: how to use sharded storage

## Introduction

For this example, we will be using DNA data to illustrate sharded storage with `seqp`.

We will:

1. Download a DNA data file in [FASTA format](https://en.wikipedia.org/wiki/FASTA_format).
2. Parse the file with [biopython](http://biopython.org/DIST/docs/tutorial/Tutorial.html#htoc11).
3. Store the sequences from the file in multiple HDF5 shards, annotating each DNA sequence with its identifier.
4. Read the DNA data back.

First, we will install some python libraries...

In [1]:
pip install biopython tqdm numpy

Note: you may need to restart the kernel to use updated packages.


## Download the data and take a look at it

In [2]:
!wget 'ftp://ftp.ebi.ac.uk/pub/databases/ena/coding/release/con/fasta/rel_con_hum_r143.cds.fasta.gz' -O rel_con_hum_r143.cds.fasta.gz
!gunzip -f 'rel_con_hum_r143.cds.fasta.gz'

--2020-05-15 12:39:33--  ftp://ftp.ebi.ac.uk/pub/databases/ena/coding/release/con/fasta/rel_con_hum_r143.cds.fasta.gz
           => ‘rel_con_hum_r143.cds.fasta.gz’
Resolving ftp.ebi.ac.uk (ftp.ebi.ac.uk)... 193.62.197.74
Connecting to ftp.ebi.ac.uk (ftp.ebi.ac.uk)|193.62.197.74|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /pub/databases/ena/coding/release/con/fasta ... done.
==> SIZE rel_con_hum_r143.cds.fasta.gz ... 15668035
==> PASV ... done.    ==> RETR rel_con_hum_r143.cds.fasta.gz ... done.
Length: 15668035 (15M) (unauthoritative)

rel_con_hum_r143.cd 100%[===================>]  14.94M  7.19MB/s    in 2.1s    

2020-05-15 12:39:35 (7.19 MB/s) - ‘rel_con_hum_r143.cds.fasta.gz’ saved [15668035]



In [3]:
from Bio import SeqIO

file_name = 'rel_con_hum_r143.cds.fasta'

for k, seq_record in enumerate(SeqIO.parse(file_name, "fasta")):
    print("ID: {}".format(seq_record.id))
    print("   - Sequence: {}...{}".format(seq_record.seq[:20], seq_record.seq[-10:]))
    print("   - Lenght: {}".format(len(seq_record)))
    if k > 3:
        break

ID: ENA|EAL24309|EAL24309.1
   - Sequence: atgaagcatgtgttgaacct...aagcatgtga
   - Lenght: 192
ID: ENA|EAL24310|EAL24310.1
   - Sequence: atggaggggccactcactcc...gctgtactga
   - Lenght: 1800
ID: ENA|EAL24311|EAL24311.1
   - Sequence: atggaggggccactcactcc...gctgtactga
   - Lenght: 1692
ID: ENA|EAL24312|EAL24312.1
   - Sequence: atggacccaaggacatccag...gacctcctga
   - Lenght: 276
ID: ENA|EAL24313|EAL24313.1
   - Sequence: atggccaggcatggctgtct...agacctgtga
   - Lenght: 2082


## Read the DNA data and store it in HDF5 with `seqp` as we go

We want to use `Hdf5RecordWriter` to write DNA sequences to files. We also want to write to multiple HDF5 files, each one containing up to a maximum amount of records, so we make use of a `ShardedWriter` decorator.

Once we have our writer, we iterate over the FASTA file sequences and store them in the writer. Each nucleotide is saved as a byte-sized integer number obtained by subtracting the ASCII index of 'a' to the nucleotide letter.

Once all the sequences are written to files, we write a piece of metadata with a dictionary from the protein name and the index within the files.

In [4]:
import json
import numpy as np
from seqp.hdf5 import Hdf5RecordWriter
from seqp.record import ShardedWriter
from tqdm import tqdm

def nucleotide2num(letter: str) -> int:
    """ Converts a nucleoide letter to an integer"""
    return ord(letter.lower()) - ord('a')

protein2idx = dict()
output_file_template = "dna_example_{:02d}.hdf5"

with ShardedWriter(Hdf5RecordWriter,
                   output_file_template,
                   max_records_per_shard=5000) as writer:

    for seq_record in tqdm(SeqIO.parse(file_name, "fasta")):
        _, _, protein = seq_record.id.split('|')
        sequence = [nucleotide2num(letter) for letter in seq_record.seq]
        idx = writer.write(np.array(sequence, dtype=np.uint8))
        protein2idx[protein] = idx

    writer.add_metadata({'protein_idx': json.dumps(protein2idx)})


65183it [01:13, 888.76it/s] 


## Read the HDF5 records back

We open the HDF5 files with a `Hdf5RecordReader`. First, we read back the dictionary with the indexes of each protein sequence, and then we retrieve the sequences associated with some specific target proteins.

In [5]:
from glob import glob
import json
from seqp.hdf5 import Hdf5RecordReader

target_proteins = ['EAL24309.1', 'EAL24312.1']

def num2nucleotide(num: int) -> str:
    """ Converts an integer to a nucleoide letter"""
    return chr(num + ord('a'))

with Hdf5RecordReader(glob('dna_example_*.hdf5')) as reader:
    loaded_protein2idx = json.loads(reader.metadata('protein_idx'))
    indexes = set(reader.indexes())
    for protein in target_proteins:
        sequence = reader.retrieve(protein2idx[protein])
        sequence = "".join([num2nucleotide(n) for n in sequence.tolist()])
        print("{} : {}...{}".format(protein, sequence[:20], sequence[-10:]))

EAL24309.1 : atgaagcatgtgttgaacct...aagcatgtga
EAL24312.1 : atggacccaaggacatccag...gacctcctga


# The same but with records with fields

We just need to specify the `sequence_field` and `fields` parameters, and provide a dictionary with such keys to function `write`.

In [6]:
output_file_template = "fields_dna_example_{:02d}.hdf5"

SEQUENCE_FIELD = 'seq'
OTHER_FIELD = 'other'
FIELDS = [SEQUENCE_FIELD, OTHER_FIELD]

with ShardedWriter(Hdf5RecordWriter,
                   output_file_template,
                   sequence_field=SEQUENCE_FIELD,
                   fields=FIELDS,
                   max_records_per_shard=5000) as writer:

    for seq_record in tqdm(SeqIO.parse(file_name, "fasta")):
        _, _, protein = seq_record.id.split('|')
        sequence = [nucleotide2num(letter) for letter in seq_record.seq]
        record = {'seq': np.array(sequence, dtype=np.uint8),
                  'other': np.array(sequence[:10], dtype=np.uint8)}
        idx = writer.write(record)
        protein2idx[protein] = idx

    writer.add_metadata({'protein_idx': json.dumps(protein2idx)})

65183it [01:26, 757.52it/s]


When reading back the records, we don't need to specify the fields, just `retrieve` would return a dictionary instead of a plain `np.ndarray`.

In [7]:
with Hdf5RecordReader(glob('fields_dna_example_*.hdf5')) as reader:
    loaded_protein2idx = json.loads(reader.metadata('protein_idx'))
    indexes = set(reader.indexes())
    for protein in target_proteins:
        record = reader.retrieve(protein2idx[protein])
        sequence = record[SEQUENCE_FIELD]
        sequence = "".join([num2nucleotide(n) for n in sequence.tolist()])
        print("{} : {}...{}".format(protein, sequence[:20], sequence[-10:]))

EAL24309.1 : atgaagcatgtgttgaacct...aagcatgtga
EAL24312.1 : atggacccaaggacatccag...gacctcctga
